## **Trabajo Práctico Individual n° 2: Aprendizaje Supervisado**
____
__Universidad Tecnológica Nacional, Buenos Aires__\
__Ingeniería Industrial__\
__Ciencia de Datos - Curso I5521__
____

- Nombre: Marco
- Apellido: Salvadó
- Legajo: 167148.0

En este trabajo práctico aplicaremos algunas técnicas aprendidas para el aprendizaje automático supervisado.

Utilizaremos un dataset de airbnb donde buscaremos predecir el precio de las noches de cada hogar en funcion de algunas variables.

#### Variables:
- Unnamed: 0: Variable de contador de registros (no será utilizada en nuetro análisis)
- neighbourhood: Barrio donde se encuentra ubicado la casa.
- room_type: Tipo de hogar en alquiler.
- price: Precio por noche.
- minimum_nights: Cantidad minimas de noches que pueden alquilar el hogar.
- number_of_reviews: Cantidad de reviews
- reviews_per_month: Tasa de reviews que tienen por mes
- calculated_host_listings_count: Cantidad de hogares ofrecidos por el host.
- availability_365: Cantidad de dias que se encuentra en alquiler el hogar

## **Objetivos del trabajo practico**
- resolver los requisitos que se detallan en las proximas celdas
- dejar comentarios # en cada linea de codigo que se agregue
- fecha limite de entrega: 13 de Octubre 2022 a las 23h59

In [1]:
## Importo librerías de maniupulación de datos
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
## Importo librerías de Aprendizaje automático
from sklearn import preprocessing
from sklearn import linear_model
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR, LinearSVR
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.model_selection import GridSearchCV

##### **Objetivo 1**
Lo primero que deberan hacer será importar
- el dataset bajo en nombre "london"
- corroborar que lo hayamos cargado correctamente.

In [3]:
# Utilizando la librería Pandas se importa el dataset
london=pd.read_csv(r'C:\Users\marco\Desktop\subir a cs de datos\TP2\TP2_airbnb_london.csv')
london.head()

,Unnamed: 0,neighbourhood,room_type,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365
0,0,Haringey,Private room,35,1,131,1.71,4,262
1,1,Ealing,Private room,70,2,2,0.07,6,62
2,2,Islington,Private room,45,1,14,0.14,2,364
3,5,Tower Hamlets,Entire home/apt,102,5,7,0.07,56,295
4,6,Barnet,Private room,29,5,114,1.13,3,285


##### **Objetivo 2**
Ahora imprimir también las dimensiones del dataset y eliminar la variable "Unnamed:0" del dataset.
Volver a guardar el dataset sin esta variable bajo el nombre "london" 

In [4]:
# Imprimo dimensiones
print(london.shape)
#Elimino la variable Unnamed
london = london.drop(['Unnamed: 0'],axis=1)
#Imprimo dimensiones nuevamente para compobar
print(london.shape)

(51827, 9)
(51827, 8)


##### **Objetivo 3**
Corroborar que el dataset no cuenta con valores nulos, si los hubiera llenarlos todos con 0.

In [5]:
london.isnull().sum().sort_values(ascending=False)

neighbourhood                     0
room_type                         0
price                             0
minimum_nights                    0
number_of_reviews                 0
reviews_per_month                 0
calculated_host_listings_count    0
availability_365                  0
dtype: int64

##### **Objetivo 4**
Mostrar el tipo de dato para las variables "neighbourhood" y "room_type".

In [6]:
print('El tipo de dato para neighbourhood es:') 
print(type('neighbourhood')) 

print('El tipo de dato para room_type es:') 
print(type('room_type'))

El tipo de dato para neighbourhood es:
<class 'str'>
El tipo de dato para room_type es:
<class 'str'>


##### **Objetivo 5**
Generar las variables dummies para estas dos variables categóricas y unirlas al dataset london.


In [7]:
# Genero las dummies para la variable neighbourhood, dentro de la variable neigh_dummies.
neigh_dummies = pd.get_dummies(london['neighbourhood'])

Imprimir las dimensiones y primeras lineas de estas dummies

In [8]:
# Imprimo dimensiones para neighbourhood
print(neigh_dummies.shape)
#Primeras líneas de Neighbourhood dummies
neigh_dummies.head()

(51827, 33)


,Barking and Dagenham,Barnet,Bexley,Brent,Bromley,Camden,City of London,Croydon,Ealing,Enfield,...,Merton,Newham,Redbridge,Richmond upon Thames,Southwark,Sutton,Tower Hamlets,Waltham Forest,Wandsworth,Westminster
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
4,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
# Genero las dummies para la variable room_tpye, dentro de la variable room_dumies.
room_dummies = pd.get_dummies(london['room_type'])

In [10]:
# Imprimo dimensiones para room_type
print(room_dummies.shape)
#Primeras líneas de room_type dummies
room_dummies.head()

(51827, 3)


,Entire home/apt,Private room,Shared room
0,0,1,0
1,0,1,0
2,0,1,0
3,1,0,0
4,0,1,0


Joinear las variables dummies al dataset



In [11]:
london = pd.concat([london, room_dummies,neigh_dummies],axis=1)
london.head()

,neighbourhood,room_type,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365,Entire home/apt,Private room,...,Merton,Newham,Redbridge,Richmond upon Thames,Southwark,Sutton,Tower Hamlets,Waltham Forest,Wandsworth,Westminster
0,Haringey,Private room,35,1,131,1.71,4,262,0,1,...,0,0,0,0,0,0,0,0,0,0
1,Ealing,Private room,70,2,2,0.07,6,62,0,1,...,0,0,0,0,0,0,0,0,0,0
2,Islington,Private room,45,1,14,0.14,2,364,0,1,...,0,0,0,0,0,0,0,0,0,0
3,Tower Hamlets,Entire home/apt,102,5,7,0.07,56,295,1,0,...,0,0,0,0,0,0,1,0,0,0
4,Barnet,Private room,29,5,114,1.13,3,285,0,1,...,0,0,0,0,0,0,0,0,0,0


In [12]:
#se tomaron una fraccion de muestras random para poder fitear el SVM en mi CPU.
london=london.sample(frac = 0.15)

##### Dividiremos nuestro dataset en la variable dependiente "y" (label o etiqueta) y las variables independientes "x"

In [13]:
Y = np.array(london[["price"]])
X = london.drop(['price', 'neighbourhood','room_type'], axis=1)

##### **Objetivo 7**

Dividir el dataset entre Train y Test, tomar un test_size del 30%.

In [14]:
# Separar train y test
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=4) 
print(x_train.shape)
print(y_train.shape)

(5441, 41)
(5441, 1)


In [15]:
x_train.head()

,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365,Entire home/apt,Private room,Shared room,Barking and Dagenham,Barnet,...,Merton,Newham,Redbridge,Richmond upon Thames,Southwark,Sutton,Tower Hamlets,Waltham Forest,Wandsworth,Westminster
15117,1,3,0.12,1,363,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5421,5,16,0.38,2,356,0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0
23535,5,1,0.05,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
18548,1,1,0.04,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
17119,2,54,2.09,1,132,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
#Separo dummies 
x_train_no_dummies=x_train.iloc[:, 0:5] # Primeras cinco columnas

x_test_no_dummies=x_test.iloc[:, 0:5] # Primeras cinco columnas


In [17]:
#variables reales
x_train_dummies=x_train.iloc[:, 5:]
x_test_dummies=x_test.iloc[:, 5:]

In [18]:
x_test_dummies.head()

,Entire home/apt,Private room,Shared room,Barking and Dagenham,Barnet,Bexley,Brent,Bromley,Camden,City of London,...,Merton,Newham,Redbridge,Richmond upon Thames,Southwark,Sutton,Tower Hamlets,Waltham Forest,Wandsworth,Westminster
31881,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
21543,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
27182,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2369,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
19496,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


##### **Objetivo 8**
Con el StandardScaler generado en la linea siguiente, realizar el transform para el xtrain y xtest guardandolos en nuevas variables llamadas "xtrain_scal" y "xtest_scal".


In [19]:
scaler = preprocessing.StandardScaler().fit(x_train_no_dummies)

In [20]:
xtrain_scal = scaler.transform(x_train_no_dummies)

In [21]:
xtest_scal = scaler.transform(x_test_no_dummies)

In [22]:
#Se vuelven a concatenar los dummies
xtrain_scal = np.concatenate([xtrain_scal, x_train_dummies],axis=1)
xtest_scal = np.concatenate([xtest_scal, x_test_dummies],axis=1)

##### **Objetivo 9**
Crear un modelo lineal y entrenarlo para nuestro dataset.

Obtener las predicciones y el "mean_squared_error" para el test set.

In [23]:
#HAGO REGRESON LINEAL Y KNEIGHBOR
#REGRISION L:
lm = LinearRegression()

#KNEIGH:
from sklearn.neighbors import KNeighborsRegressor
knn = KNeighborsRegressor(weights = "distance")
parameters_k = np.arange(20,41,5)
parameters_knn = [{'n_neighbors': parameters_k}]
regressor_knn = GridSearchCV(knn, parameters_knn, refit = True, cv=5, verbose=True)

In [24]:
#FITEO Y GUARDO PREDICCION EN VARIABLE y_pred
#REGRISION L:
lm.fit(xtrain_scal, y_train)
y_predl = lm.predict(xtest_scal)
#KNEIGH:
regressor_knn.fit(xtrain_scal, y_train)
y_predk = regressor_knn.predict(xtest_scal)

Fitting 5 folds for each of 5 candidates, totalling 25 fits


In [25]:
#Evaluacion del modelo
print(" REGRISION L:")
from sklearn import metrics
print('MSE regresion LINEAL:',metrics.mean_squared_error(y_test,y_predl))

print("\n KNEIGH:")
print(regressor_knn.best_score_)
print(regressor_knn.best_params_)
ypred2 = regressor_knn.predict(xtest_scal)
print('MSE regresion LINEAL:',np.sqrt(mean_squared_error(y_test,y_predk)))



 REGRISION L:
MSE regresion LINEAL: 1589.779756730862

 KNEIGH:
0.48802828899410056
{'n_neighbors': 40}
MSE regresion LINEAL: 40.18599013087971


##### **Objetivo 10**
Crear un SVM Regressor y un GridSearch para realizar la búsqueda de hiperparámetros.

Asignar dos posibles valores para el parámetro "kernel", y tres posibles valores para "C" y "gamma".

Setear al parámetro "refit" en True y hacer 5 CrossValidation.

In [26]:
svreg = SVR()

In [27]:
param_svreg = {'kernel':('linear', 'rbf'), 'C':[1, 10,100 ], 'gamma':[ 0.1,1,10]}

In [28]:
regressor_svr = GridSearchCV(estimator = svreg, param_grid = param_svreg, cv = 5, n_jobs = -1, verbose = True, refit = True)

In [29]:
regressor_svr.fit(xtrain_scal, y_train.ravel())

Fitting 5 folds for each of 18 candidates, totalling 90 fits


GridSearchCV(cv=5, estimator=SVR(), n_jobs=-1,
             param_grid={'C': [1, 10, 100], 'gamma': [0.1, 1, 10],
                         'kernel': ('linear', 'rbf')},
             verbose=True)

##### **Objetivo 11**

Mostrar cuales fueron los mejores hiperparámetros y para estos obtener el mean_squared_error en el test set.

In [30]:
regressor_svr.best_params_

{'C': 100, 'gamma': 0.1, 'kernel': 'rbf'}

In [31]:
y_test_predSVR = regressor_svr.predict(xtest_scal)
y_test_predSVR

array([31.50715742, 36.9514498 , 36.05650358, ..., 89.52461451,
       44.65957228, 24.38477096])

In [32]:
np.sqrt(mean_squared_error(y_test, y_test_predSVR))
y_test_predSVR = regressor_svr.predict(xtest_scal)
# Model Evaluation
acc_svm = metrics.r2_score(y_test, y_test_predSVR)
print('R^2:', acc_svm)
print('Adjusted R^2:',1 - (1-metrics.r2_score(y_test, y_test_predSVR))*(len(y_test)-1)/(len(y_test)-xtest_scal.shape[1]-1))
print('MAE:',metrics.mean_absolute_error(y_test, y_test_predSVR))
print('MSE:',metrics.mean_squared_error(y_test, y_test_predSVR))
print('RMSE:',np.sqrt(metrics.mean_squared_error(y_test, y_test_predSVR)))

R^2: 0.4975272937164572
Adjusted R^2: 0.48853498426310704
MAE: 26.951837753416964
MSE: 1638.450441695863
RMSE: 40.47777713382818


##### **Objetivo 12**
Repetir el objetivo 10 pero ahora seteando nuevos valores para los hiperparámetros mencionados con el objetivo de mejorar las predicciones.

Cuales son los mejores hiperparámetros ahora?
Obtenemos una mejora en las predicciones?
Mostrar los resultados.

In [33]:
svreg = SVR()

In [34]:
param_svreg = {'kernel':('linear', 'rbf'), 'C':[80, 100,120 ], 'gamma':[ 0.005,0.01,0.1]}

In [35]:
regressor_svr = GridSearchCV(estimator = svreg, param_grid = param_svreg, cv = 5, n_jobs = -1, verbose = True, refit = True)

In [36]:
regressor_svr.fit(xtrain_scal, y_train.ravel())

Fitting 5 folds for each of 18 candidates, totalling 90 fits


GridSearchCV(cv=5, estimator=SVR(), n_jobs=-1,
             param_grid={'C': [80, 100, 120], 'gamma': [0.005, 0.01, 0.1],
                         'kernel': ('linear', 'rbf')},
             verbose=True)

In [37]:
regressor_svr.best_params_

{'C': 80, 'gamma': 0.1, 'kernel': 'rbf'}

In [38]:
y_test_predSVR = regressor_svr.predict(xtest_scal)
y_test_predSVR

array([31.92701465, 36.35518307, 36.01223206, ..., 90.00066485,
       45.47390059, 24.2869732 ])

In [39]:
np.sqrt(mean_squared_error(y_test, y_test_predSVR))
y_test_predSVR = regressor_svr.predict(xtest_scal)
# Model Evaluation
acc_svm = metrics.r2_score(y_test, y_test_predSVR)
print('R^2:', acc_svm)
print('Adjusted R^2:',1 - (1-metrics.r2_score(y_test, y_test_predSVR))*(len(y_test)-1)/(len(y_test)-xtest_scal.shape[1]-1))
print('MAE:',metrics.mean_absolute_error(y_test, y_test_predSVR))
print('MSE:',metrics.mean_squared_error(y_test, y_test_predSVR))
print('RMSE:',np.sqrt(metrics.mean_squared_error(y_test, y_test_predSVR)))

R^2: 0.4979145846117117
Adjusted R^2: 0.4889292061608518
MAE: 26.914203740631095
MSE: 1637.187573224682
RMSE: 40.462174598316906
